# Competition

[G-Research Crypto Forecasting](https://www.kaggle.com/c/g-research-crypto-forecasting)

# Overview

14種類の暗号通貨の短期的なリターンを予測する。

毎日、400億ドル以上の暗号通貨が取引されています。<br>
暗号通貨は、投機や投資の対象として最も人気のある資産の一つですが、その変動の激しさは証明されています。<br>
急激に変動する価格は、幸運な少数の人々を億万長者にし、他の人々に大きな損失をもたらしています。<br>
これらの価格変動の一部は、事前に予測できたのでしょうか？

このコンペティションでは、機械学習の専門知識を使って、人気のある14種類の暗号通貨の短期的なリターンを予測します。<br>
私たちは、2018年までさかのぼって数百万行の高頻度市場データのデータセットを蓄積しており、これを使ってモデルを構築していただきます。<br>
提出期限が過ぎると、収集されたライブの暗号データを使用して、その後3カ月かけて最終的なスコアが計算されます。

何千人ものトレーダーが同時に活動することで、ほとんどのシグナルは一過性のものとなり、<br>
持続的なアルファを見つけることは非常に難しく、オーバーフィッティングの危険性もかなり高くなります。<br>
また、2018年以降、暗号市場への関心が爆発的に高まっているため、データのボラティリティと相関構造は高度な非定常性を持つ可能性があります。<br>
成功した参加者は、これらの検討事項に注意深く注意を払い、その過程で金融予測の技術と科学について貴重な洞察を得ることになるでしょう。

G-Research社は、ヨーロッパを代表するクオンツ・ファイナンスのリサーチ会社です。<br>
私たちは、機械学習やビッグデータ、そして最先端のテクノロジーを駆使して、市場予測の可能性を長年にわたって追求してきました。<br>
労働者向けのデータサイエンスとAIの教育を専門とするケンブリッジ・スパークは、G-Research社と提携してこのコンペティションを開催しています。

# Evaluation

投稿された内容は、ピアソン相関係数の加重バージョンで評価されます。<br>
詳細は、このチュートリアルノートの「予測の詳細と評価」のセクションをご覧ください。<br>
このコンペティションでは、提供されているPython時系列APIを使用して投稿する必要があります。<br>
これにより、モデルが時間的に先に進むことはありません。<br>
API を使用するには、Kaggle Notebooks のこのテンプレートに従ってください。

In [ ]:
# import gresearch_crypto
# env = gresearch_crypto.make_env()
# iter_test = env.iter_test()
# for (test_df, sample_prediction_df) in iter_test:
    # sample_prediction_df['Target'] = 0
    # env.predict(sample_prediction_df)

# Timeline

このコンテストは、トレーニング期間と、実際の市場データを使ってモデルを実行する期間がある予測コンテストです。

トレーニングスケジュール<br>
・2021年11月2日 - 開始日<br>
・2022年1月25日 - エントリー締め切り。競技に参加するためには、この日までに競技規則に同意する必要があります。<br>
・2022年1月25日 - チーム合併期限。参加者がチームに参加したり、チームを合併したりできる最終日です。<br>
・2022年2月1日 - 最終提出期限。

すべての締め切りは、特に明記されていない限り、該当日の午後11時59分（UTC）です。<br>
**日本時間では、2022年02月02日(火) 08:59（JTC）**となります。<br>
大会主催者が必要と判断した場合は、コンテストのタイムラインを更新する権利を有します。

予測タイムライン<br>
最終提出期限の後、選択されたノートブックで実行される市場データの更新を反映して、<br>
リーダーボードが定期的に更新されます。更新はおよそ2週間ごとに行われます。

・2022年5月3日 - コンテスト終了日 - 受賞者の発表

# Module

In [ ]:
import time
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime as dt

In [ ]:
pd.set_option('display.max_colwidth', 150)

In [ ]:
sns.set()

# Datasets

### train

In [ ]:
train = pd.read_csv('../input/g-research-crypto-forecasting/train.csv') 
print(train.shape)
train.head(10)

In [ ]:
pd.DataFrame([['timestamp', 'タイムスタンプ', '秒単位のUnixタイムスタンプ', 'データ期間（2018/01/01 09:01:00〜2021/09/21 09:00:00）'],
              ['Asset_ID', '暗号通貨に対応する資産ID', 'asset_detailsに対応している', ''],
              ['count', '直前の1分間の取引数', '', ''],
              ['Open', 'その1分間の始値', 'データは1分ごとの各暗号資産のデータが記載されている', ''],
              ['High', 'その1分間に到達した最高価格', '', ''],
              ['Low', 'その1分間に到達した最低価格', '', ''],
              ['Close', 'その1分間の終値', '', ''],
              ['Volume', 'その1分間に売買された資産の数量', '', ''],
              ['VMAP', 'その1分間での資産の平均価格を出来高で加重したもの', '', ''],
              ['TARGET', '15分間での暗号資産の残余対数リターン', '', '']],
              columns=['カラム', '意味', '補足1', '補足2'])

### asset_details

In [ ]:
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv') 
asset_details = asset_details.sort_values("Asset_ID").reset_index(drop=True)
print(asset_details.shape)
asset_details.head(10)

In [ ]:
pd.DataFrame([['Asset_id', '暗号通貨に対応する資産ID', '', ''],
              ['Weight', '重み', '', ''],
              ['Asset_name', '暗号資産名', '', '']],
              columns=['カラム', '意味', '補足1', '補足2'])

### example_test

In [ ]:
example_test = pd.read_csv('../input/g-research-crypto-forecasting/example_test.csv')
print(example_test.shape)
example_test.head(10)

In [ ]:
pd.DataFrame([['timestamp', 'タイムスタンプ', '秒単位のUnixタイムスタンプ', 'データ期間（2018/01/01 09:01:00〜2021/09/21 09:00:00）'],
              ['Asset_ID', '暗号通貨に対応する資産ID', 'asset_detailsに対応している', ''],
              ['count', '直前の1分間の取引数', '', ''],
              ['Open', 'その1分間の始値', 'データは1分ごとの各暗号資産のデータが記載されている', ''],
              ['High', 'その1分間に到達した最高価格', '', ''],
              ['Low', 'その1分間に到達した最低価格', '', ''],
              ['Close', 'その1分間の終値', '', ''],
              ['Volume', 'その1分間に売買された資産の数量', '', ''],
              ['VMAP', 'その1分間での資産の平均価格を出来高で加重したもの', '', ''],
              ['group_num', 'グループの番号', '', ''],
              ['row_id', '行ID', '', '']],
              columns=['カラム', '意味', '補足1', '補足2'])

### example_sumple_submission

In [ ]:
example_sumple_submission = pd.read_csv('../input/g-research-crypto-forecasting/example_sample_submission.csv')
print(example_sumple_submission.shape)
example_sumple_submission.head(10)

In [ ]:
pd.DataFrame([['group_num', 'グループ番号', '', ''],
              ['row_id', '行ID', '', ''],
              ['TARGET', '15分間での暗号資産の残余対数リターン', '', '']],
              columns=['カラム', '意味', '補足1', '補足2'])

# Info

### train

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

### asset_details

In [ ]:
asset_details.dtypes

In [ ]:
asset_details.isnull().sum()

In [ ]:
asset_details.describe()

# EDA

UNIXタイムスタンプは、以下の時間を起点としてカウントしたものである。

In [ ]:
zero = datetime.datetime.fromtimestamp(0)
print(zero)

学習データの期間を確認する。

In [ ]:
start = datetime.datetime.fromtimestamp(1514764860)
end = datetime.datetime.fromtimestamp(1632182400)
print('start:', start)
print('end:', end)

UNIXタイムスタンプは60増えると1分、3600で1時間の経過となる。

In [ ]:
x = 1514764860
t1 = datetime.datetime.fromtimestamp(x)
t2 = datetime.datetime.fromtimestamp(x+60)
t3 = datetime.datetime.fromtimestamp(x+3600)

print(t1)
print(t2)
print(t3)

暗号資産別に見ると、UNIXタイムスタンプが60ずつ増えている。<br>
つまり、各暗号資産のデータが1分ごとに記録されている。

In [ ]:
train.query('Asset_ID == 1').head().reset_index(drop=True)

暗号資産ごとの学習データ数を確認する。

In [ ]:
asset_count= []
for i in range(14):
    count = (train["Asset_ID"]==i).sum()
    asset_count.append(count)
fig = px.bar(x = asset_details.sort_values("Asset_ID")["Asset_Name"],
             y = asset_count , 
             color = asset_count ,
             color_continuous_scale="Emrld")
fig.update_xaxes(title="Assets")
fig.update_yaxes(title = "Number of Rows")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

暗号資産ごとの価格変動を見ていく。

In [ ]:
def crypto_df(asset_id ,data=train):
    df = train[train["Asset_ID"]==asset_id].reset_index(drop = True)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.set_index('timestamp')
    return df

btc = crypto_df(asset_id = 1)
eth = crypto_df(asset_id = 6)
ltc = crypto_df(asset_id = 9)

In [ ]:
def candelstick_chart(data,title):
    candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                               open = data[('Open')], 
                                               high = data[('High')], 
                                               low = data[('Low')], 
                                               close = data[('Close')])])
    candlestick.update_xaxes(title_text = 'Time',
                             rangeslider_visible = True)

    candlestick.update_layout(
    title = {
        'text': '{:} Candelstick Chart'.format(title),
        'y':0.90,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

    candlestick.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return candlestick

暗号資産（ビットコイン）の価格変動を確認する。

In [ ]:
btc_plot = candelstick_chart(btc[-2500:],title = "Bitcoin(BTC)")
btc_plot.show()

暗号資産（ライトコイン）の価格変動を確認する。

In [ ]:
ltc_plot = candelstick_chart(ltc[-2500:],title = "Litecoin(LTC)")
ltc_plot.show()

暗号資産（イーサリアム）の価格変動を確認する。

In [ ]:
eth_plot = candelstick_chart(eth[-2500:],title = "Ethereum(ETH)")
eth_plot.show()

OHLCは、Open・High・Low・Closeのチャートで、各財務データの始値、高値、安値および終値を表す。<br>
ファイナンシャルシナリオと株の変動の分析のために役立てることができる。

暗号資産ごとのOHLCチャートを見ていく。

In [ ]:
def ohlc_chart(data,title):
    ohlc = go.Figure(data = [go.Ohlc(x =data.index, 
                                               open = data[('Open')], 
                                               high = data[('High')], 
                                               low = data[('Low')], 
                                               close = data[('Close')])])
    ohlc.update_xaxes(title_text = 'Time',
                             rangeslider_visible = True)

    ohlc.update_layout(
    title = {
        'text': '{:} OHLC Chart'.format(title),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

    ohlc.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return ohlc

暗号資産（ビットコイン）のOHLCチャートを確認する。

In [ ]:
ohlc_chart(btc[:200], title = "Bitcoin(BTC)")

暗号資産（ライトコイン）のOHLCチャートを確認する。

In [ ]:
ohlc_chart(ltc[-100:], title = "Litecoin(LTC)")

暗号資産（イーサリアム）のOHLCチャートを確認する。

In [ ]:
ohlc_chart(eth[-100:], title = "Ethereum(ETH)")

暗号資産ごとの取引量の推移を見ていく。

In [ ]:
def vol_traded(data ,title,color):
    area = px.area(data_frame=data,
               x = data.index ,
               y = "Volume",
               markers = True)
    area.update_traces(line_color=color)
    area.update_xaxes(
        title_text = 'Time',
        rangeslider_visible = True)
    area.update_yaxes(title_text = 'Number of trades every minute')
    area.update_layout(showlegend = True,
        title = {
            'text': '{:} Volume Traded'.format(title),
            'y':0.94,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    return area

暗号資産（ビットコイン）の取引量を確認する。

In [ ]:
vol_traded(btc[-300:], "Bitcoin(BTC)",color = "Yellow")

暗号資産（ライトコイン）の取引量を確認する。

In [ ]:
vol_traded(ltc[-300:], "Litecoin(LTC)",color = "Blue")

暗号資産（イーサリアム）の取引量を確認する。

In [ ]:
vol_traded(eth[-50:], "Ethereum (ETH)",color = "Red")

暗号資産（イーサリアム）のデータについて見る。

In [ ]:
eth = train[train["Asset_ID"]==6].set_index("timestamp")
eth.info(show_counts =True)

目的変数（Target）に欠損値があることを確認できる。

In [ ]:
eth.isnull().sum()

In [ ]:
btc = train[train["Asset_ID"]==1].set_index("timestamp")
btc.head()

timestampをdatetime型に変換して、データの時間範囲を確認する。

In [ ]:
beg_btc = btc.index[0].astype('datetime64[s]')
end_btc = btc.index[-1].astype('datetime64[s]')
beg_eth = eth.index[0].astype('datetime64[s]')
end_eth = eth.index[-1].astype('datetime64[s]')

print('BTC data goes from ', beg_btc, 'to ', end_btc)
print('Ethereum data goes from ', beg_eth, 'to ', end_eth)

分単位の資産データの欠損は、NaNデータではなく、行自体がないことで表される。<br>
そのため、ある行と1つ前の行との差を取って、それが60分でなければ、欠損があると分かる。

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

欠損箇所を埋めるために、直前の値で補完する

In [ ]:
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')
(eth.index[1:]-eth.index[:-1]).value_counts().head()

暗号資産（ビットコインとイーサリアム）の時間推移を確認する。

In [ ]:
f = plt.figure(figsize=(15,4))

btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')

ax = f.add_subplot(121)
plt.plot(btc['Close'], label='BTC')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin')

ax2 = f.add_subplot(122)
ax2.plot(eth['Close'], color='red', label='ETH')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum')

plt.tight_layout()
plt.show()

ビットコインとイーサリアムの終値を比較し、両者に相関関係があるかを見る。

In [ ]:
totimestamp = lambda s: np.int32(time.mktime(dt.strptime(s, "%d/%m/%Y").timetuple()))

btc_mini_2021 = btc.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]
eth_mini_2021 = eth.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]

In [ ]:
f = plt.figure(figsize=(7,8))

ax = f.add_subplot(211)
plt.plot(btc_mini_2021['Close'], label='btc')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin Close')

ax2 = f.add_subplot(212)
ax2.plot(eth_mini_2021['Close'], color='red', label='eth')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum Close')

plt.tight_layout()
plt.show()

さらに短い期間で見ると、潜在的な相関関係が分かる。<br>
より細かく理解するには、各資産のリターンを計算するのが良い。

ある資産の価格変動を分析するためには、価格差を扱えば良いが<br>
資産によって価格スケールが異なるため、リターンを比較することは容易ではない。<br>
これは、価格の変化率（リターン）を計算することで解決することができる。<br>
また、このリターンは投資資金の変化率と一致する。

In [ ]:
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

2つの資産（ビットコインとイーサリアム）の対数リターンを可視化する。

In [ ]:
lret_btc = log_return(btc_mini_2021.Close)[1:]
lret_eth = log_return(eth_mini_2021.Close)[1:]
lret_btc.rename('lret_btc', inplace=True)
lret_eth.rename('lret_eth', inplace=True)

plt.figure(figsize=(8,4))
plt.plot(lret_btc);
plt.plot(lret_eth);
plt.show()

2021年における、2つの資産（ビットコインとイーサリアム）の相関関係の変化を見る。

In [ ]:
lret_btc_long = log_return(btc.Close)[1:]
lret_eth_long = log_return(eth.Close)[1:]
lret_btc_long.rename('lret_btc', inplace=True)
lret_eth_long.rename('lret_eth', inplace=True)
two_assets = pd.concat([lret_btc_long, lret_eth_long], axis=1)

corr_time = two_assets.groupby(two_assets.index//(10000*60)).corr().loc[:,"lret_btc"].loc[:,"lret_eth"]

corr_time.plot();
plt.xticks([])
plt.ylabel("Correlation")
plt.title("Correlation between BTC and ETH over time");

資産間の相関は高いが、変動しており、非定常性があることに気をつけなければならない。

相関行列を可視化することで、すべての資産間の相関を確認する。<br>
いくつかの資産は、他の資産よりもはるかに高いペアワイズ相関を持っていることに注意する必要がある。

In [ ]:
all_assets_2021 = pd.DataFrame([])
for asset_id, asset_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):
  asset = train[train["Asset_ID"]==asset_id].set_index("timestamp")
  asset = asset.loc[totimestamp('01/01/2021'):totimestamp('01/05/2021')]
  asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
  lret = log_return(asset.Close.fillna(0))[1:]
  all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")

In [ ]:
plt.imshow(all_assets_2021.corr());
plt.yticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values);
plt.xticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values, rotation='vertical');
plt.colorbar();

# Reset

Notebookのカーネルをリセットする。

In [ ]:
%reset -f

# Module

In [ ]:
import gc
import time
import optuna
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import gresearch_crypto
import scipy.stats as stats
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
pd.set_option('display.max_colwidth', 150)

In [ ]:
sns.set()

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

# Datasets

In [ ]:
df_train = pd.read_csv('../input/g-research-crypto-forecasting/train.csv') 
print(df_train.shape)
df_train.head(10)

In [ ]:
pd.DataFrame([['timestamp', 'タイムスタンプ', '秒単位のUnixタイムスタンプ', 'データ期間（2018/01/01 09:01:00〜2021/09/21 09:00:00）'],
              ['Asset_ID', '暗号通貨に対応する資産ID', 'asset_detailsに対応している', ''],
              ['count', '直前の1分間の取引数', '', ''],
              ['Open', 'その1分間の始値', 'データは1分ごとの各暗号資産のデータが記載されている', ''],
              ['High', 'その1分間に到達した最高価格', '', ''],
              ['Low', 'その1分間に到達した最低価格', '', ''],
              ['Close', 'その1分間の終値', '', ''],
              ['Volume', 'その1分間に売買された資産の数量', '', ''],
              ['VMAP', 'その1分間での資産の平均価格を出来高で加重したもの', '', ''],
              ['TARGET', '15分間での暗号資産の残余対数リターン', '', '']],
              columns=['カラム', '意味', '補足1', '補足2'])

In [ ]:
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv') 
asset_details = asset_details.sort_values("Asset_ID").reset_index(drop=True)
print(asset_details.shape)
asset_details.head(10)

In [ ]:
pd.DataFrame([['Asset_id', '暗号通貨に対応する資産ID', 'trainとの結合キーになっている', ''],
              ['Weight', '重み', '', ''],
              ['Asset_name', '暗号資産名', '', '']],
              columns=['カラム', '意味', '補足1', '補足2'])

# Caution

テストデータは元の学習データに含まれているので、公開されているNotebookのLBスコアはオーバーフィッティングしているものも多い。<br>
オーバーフィッティングなしのベストLBは、今日現在で0.016となっている。これはさらに改善することができる。

# Pipeline

学習データにテストデータも含まれてしまっているので、除去する。

In [ ]:
REMOVE_LB_TEST_OVERLAPPING_DATA = True

if REMOVE_LB_TEST_OVERLAPPING_DATA:
    df_train['datetime'] = pd.to_datetime(df_train['timestamp'], unit='s')
    df_train = df_train[df_train['datetime'] < '2021-06-13 00:00:00']

print(df_train.shape)
df_train.head()

前処理を行い、暗号資産ごとにモデリングをする。

In [ ]:
# def hlco_ratio(df): 
    # return (df['High']-df['Low'])/(df['Close']-df['Open'])

def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    # df_feat['hlco_ration'] = hlco_ratio(df_feat)
    return df_feat

In [ ]:
def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
    
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    model = LGBMRegressor(n_estimators=10)
    model.fit(X, y)
    return X, y, model

暗号資産ごとに X, y, model を格納していく。

In [ ]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(asset_details['Asset_ID'], asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = get_Xy_and_model_for_asset(df_train, asset_id)
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

# Prediction

複数のモデルを、暗号資産ごとに使い分けて予測をする。<br>
df_pred には、sample_prediction が入っている。

In [ ]:
all_df_test = []

for i, (df_test, df_pred) in enumerate(iter_test):
    for j, row in df_test.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_features(row)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        if i == 0 and j == 0:
            display(x_test)

    if i == 0:
        display(df_pred)
    all_df_test.append(df_test)

    env.predict(df_pred)

# Overlap

学習データとテストデータで、重複している期間がないかを確認する。

In [ ]:
df_test = pd.concat(all_df_test)
df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
df_train['datetime'] = pd.to_datetime(df_train['timestamp'], unit='s')

In [ ]:
print('Train End:', df_train['datetime'].max())
print('Test Start:', df_test['datetime'].max())

# References

### EDA

・[Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)<br>
・[G-Research Plots + EDA](https://www.kaggle.com/odins0n/g-research-plots-eda)<br>

### Pipeline

・[G-Research- Starter LGBM Pipeline](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline)<br>
・[G-Research- Using the overlap fully [LB=0.99]](https://www.kaggle.com/icaram/g-research-using-the-overlap-fully-lb-0-99)<br>
・[G-Research Crypto - XGB lag](https://www.kaggle.com/mingyangyi/g-research-crypto-xgb-lag)